# AML Coursework assessment

## Data Exploration

### Wisconsin Diagnostic Breast Cancer Dataset

The Wisconsin Diagnostic Breast Cancer dataset describes the measurements on cells in suspicious lumps in a women's breast. This dataset, obtained from the University of Wisconsin Hospitals, includes 30 feature variables and 1 target variable. 

The features, or input attributes, are numeric, and a binary target variable: M for Malignant and B for benign.

Each cell-nucleus include 10 actual features:

- Radius (mean of distances from center to points on the perimeter)
- Texture (standard deviation of gray-scale values)
- Perimeter
- Area
- Smoothness (local variation in radius lengths)
- Compactness (perimeter^2 / area - 1.0)
- Concavity (severity of concave portions of the contour)
- Concave points (number of concave portions of the contour)
- Symmetry 
- Fractal dimension ("coastline approximation" - 1)

Now, we will load the dataset so we can check the data in detail. To do so, we'll install the required libraries and packages.

In [ ]:
!pip3 install matplotlib pandas numpy

In [ ]:
!pip3 install -U ucimlrepo

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

data_combined = pd.concat([X, y], axis=1)
  
# metadata 
#print(breast_cancer_wisconsin_diagnostic.metadata) 
  
# variable information 
#print(breast_cancer_wisconsin_diagnostic.variables)



### Checking the data

We'll retrieve a data preview of the data so we can have a first look at it.

In [ ]:
# Print the first few rows of the data
print("\nFirst few rows of Features and Target combined")
print(data_combined.head())

### The dimensionality of the dataset

We can see that we're able to access the repository from UCIML and the data is loaded. Now we need to understand the dimensionality of the dataset in terms of number of rows, columns (features and target variable).

In [ ]:
# Print the shape of the data

print("Shape of X (features):", X.shape)
print("Shape of y (targets):", y.shape)

### Statistical Summary

Let's retrieve the statistical properties of each attribute in order to get some insights.

In [ ]:
pd.set_option('display.width', 100)
pd.set_option('display.precision', 3)

descriptionDataCombined = data_combined.describe()

print(descriptionDataCombined)

### Class Distribution

As this is a classification problem, we need to analyse the data in order to check how many observations do we have for each class. If they are highly imbalanced, then we'll need to take further actions in the data pre-processing.

In [ ]:
class_counts = data_combined.groupby('Diagnosis').size()
print(class_counts)

total = class_counts.sum()
proportion_B = class_counts['B'] / total
proportion_M = class_counts['M'] / total

print(f"\nProportion of class B: {proportion_B:.2f}")
print(f"Proportion of class M: {proportion_M:.2f}")

### Features Correlation 

By calculating the correlation between the different variables we'll be able to analyse if they are tightly coupled (highly correlated). This is important as it can affect the performance of the machine learning algorithms.

Let's calculate the Pearson's Correlation Coefficient, assuming that we have a normal distribution in the attributes. Before that, we need to convert the target variable to a numeric one.

In [ ]:
# Convert the Diagnosis column from string to numerical values.
data_combined['Diagnosis'] = data_combined['Diagnosis'].map({'M': 1, 'B': 0})

# Calculate the Pearson's Correlation Coefficient
correlations = data_combined.corr(method='pearson')
print(correlations)

As the table size is big, let's retrieve those variables that are highly correlated (> 0.75) by sorting and printing them.

In [ ]:
# Unstack the correlation matrix
correlation_pairs = correlations.unstack()

# Sort the correlation pairs
sorted_pairs = correlation_pairs.sort_values(kind="quicksort", ascending=False)

# Retrieve high correlations and exclude self-correlations, as they aren't relevant.
high_correlations = sorted_pairs[(sorted_pairs != 1.0) & (sorted_pairs.abs() > 0.75)]

print(high_correlations)

Let's plot a correlation matrix to achieve a better understanding of the data correlation

Datasource: https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas

In [ ]:
# Plotting the Correlation Matrix

plt.figure(figsize=(10, 8))
plt.imshow(correlations, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Correlation Matrix')
plt.xticks(range(len(correlations.columns)), correlations.columns, rotation=90)
plt.yticks(range(len(correlations.index)), correlations.index)
plt.show()

We can see that the highly correlated variables are the radius and perimeter. This makes sense due to its mathematical relationship, as the radius is directly proportional to the perimeter.